In [1]:
# import libraries

# for processing data tables
import numpy as np
import pandas as pd

# for visualizations
import matplotlib.pyplot as plt
import seaborn as sns

# for sending requests to url and parsing
import requests
import json

import sys
sys.path.append('../')

# import useful cumtom functions and variables from my helper.py
from helper import request_data_kosis
from helper import dict_hl_name
from helper import map_hl

import configparser
config = configparser.ConfigParser()
config.read('../../config.ini')

['../../config.ini']

## 1. City Product per Capita

In [2]:
df_cppc_hl = request_data_kosis(url = config['cppc']['hl_url'], target_period = '2020',variable_name = 'cpcc')

In [3]:
df_cppc_hl.head()

,C1,C1_NM,cpcc,period
0,11,서울특별시,45859.0,2020
1,21,부산광역시,27426.0,2020
2,22,대구광역시,23958.0,2020
3,23,인천광역시,30513.0,2020
4,24,광주광역시,27994.0,2020


## 2. Old Age Dependency Ratio

#### High-level

In [4]:
# working population = people ages 15 to 64
working_pop = ['15 - 19세',
               '20 - 24세',
               '25 - 29세', 
               '30 - 34세', 
               '35 - 39세', 
               '40 - 44세', 
               '45 - 49세', 
               '50 - 54세', 
               '55 - 59세', 
               '60 - 64세']

# old population = people ages 65 and over
old_pop = ['65 - 69세', '70 - 74세', '75 - 79세', '80세 이상']

In [5]:
df_oadr_hl = request_data_kosis(url = config['oadr']['hl_url'], target_period = '2021', index = ['C1','C1_NM'], columns = 'C3_NM', scale = 'hl')

# calculating people ages 15 to 64 and people ages 65 and over
df_oadr_hl['working_pop'] = df_oadr_hl.loc[:,working_pop].sum(axis = 1)
df_oadr_hl['old_pop'] = df_oadr_hl.loc[:,old_pop].sum(axis = 1)

# calculating old age dependency ratio
df_oadr_hl['oadr'] = df_oadr_hl['old_pop'] / df_oadr_hl['working_pop'] 

In [6]:
df_oadr_hl = df_oadr_hl.loc[:,['C1','C1_NM', 'oadr', 'period']]

In [7]:
df_oadr_hl.head()

,C1,C1_NM,oadr,period
0,11,서울특별시,0.221983,2021
1,21,부산광역시,0.284877,2021
2,22,대구광역시,0.238335,2021
3,23,인천광역시,0.195436,2021
4,24,광주광역시,0.200308,2021


#### Low-level

In [8]:
df_oadr_ll = request_data_kosis(url = config['oadr']['ll_url'], target_period = '2021', index = ['C1','C1_NM'], columns = 'C3_NM', scale = 'll')

# calculating people ages 15 to 64 and people ages 65 and over
df_oadr_ll['old_pop'] = df_oadr_ll.loc[:,old_pop].sum(axis = 1)
df_oadr_ll['working_pop'] = df_oadr_ll.loc[:,working_pop].sum(axis = 1)

# calculating old age dependency ratio
df_oadr_ll['oadr'] = df_oadr_ll['old_pop'] / df_oadr_ll['working_pop'] 

In [9]:
df_oadr_ll = df_oadr_ll.loc[:,['C1','C1_NM', 'oadr', 'period']]
df_oadr_ll = df_oadr_ll[df_oadr_ll['C1'].str.len()!=2]
df_oadr_ll['C1_hl'] = df_oadr_ll['C1'].str[:2]

In [10]:
df_oadr_ll = df_oadr_ll.merge(df_oadr_hl.loc[:,['C1','C1_NM']].rename(columns={'C1':'C1_hl','C1_NM':'hl'}), on='C1_hl', how = 'left')
df_oadr_ll = df_oadr_ll.loc[np.logical_not((df_oadr_ll['hl'].str[-1]=='도') & (df_oadr_ll['C1_NM'].str[-1]=='구')),:]

In [11]:
df_oadr_ll.head()

,C1,C1_NM,oadr,period,C1_hl,hl
0,11010,종로구,0.252811,2021,11,서울특별시
1,11020,중구,0.259984,2021,11,서울특별시
2,11030,용산구,0.223003,2021,11,서울특별시
3,11040,성동구,0.210080,2021,11,서울특별시
4,11050,광진구,0.192343,2021,11,서울특별시


## 3. Mean Household Income

## 4. Economic Density

In [17]:
df_gdrp_hl = request_data_kosis(url = config['gdrp']['hl_url'], target_period = '2020', variable_name = 'gdrp')
df_a_hl = request_data_kosis(url = config['a']['hl_url'], index = ['C1','C1_NM', 'C2', 'C2_NM'], target_period = '2020', variable_name = 'area')

# exclude the low-level regions
df_a_hl = df_a_hl.loc[(df_a_hl['C1_NM'] == df_a_hl['C2_NM']) & (df_a_hl['C2_NM'] != '전국'),:].reset_index(drop=True).drop(['C2','C2_NM'], axis = 1)

In [18]:
df_a_hl['C1_NM'] = df_a_hl['C1_NM'].apply(lambda x: map_hl(x,dict_hl_name))

In [19]:
df_gdrp_hl.merge(df_a_hl.loc[:,['C1_NM','area']], on='C1_NM')

,C1,C1_NM,gdrp,period,area
0,11,서울특별시,413669812.0,2020,6.052285e+08
1,21,부산광역시,86499417.0,2020,7.700748e+08
2,22,대구광역시,54000377.0,2020,8.834875e+08
3,23,인천광역시,85404479.0,2020,1.065226e+09
4,24,광주광역시,39214877.0,2020,5.011284e+08
5,25,대전광역시,41061819.0,2020,5.396635e+08
6,26,울산광역시,66837557.0,2020,1.062086e+09
7,29,세종특별자치시,11659372.0,2020,4.649124e+08
8,31,경기도,467281507.0,2020,1.019527e+10
9,32,강원도,45122237.0,2020,1.682968e+10


## 5. Economic Specialization

## 6. Unemployment Rate

In [26]:
# 광역지방자치단체
df_uer_hl = request_data_kosis(url = config['uer']['hl_url'], target_period = '2021', variable_name = 'uer')

# 기초지방자치단체
df_uer_ll = request_data_kosis(url = config['uer']['ll_url'], target_period = '202102', variable_name = 'uer')
df_uer_ll['period'] = '2021 2nd half'

In [27]:
print(df_uer_hl.shape)
df_uer_hl.head()

(17, 4)


,C1,C1_NM,uer,period
0,11,서울특별시,4.8,2021
1,21,부산광역시,3.4,2021
2,22,대구광역시,3.7,2021
3,23,인천광역시,4.0,2021
4,24,광주광역시,3.6,2021


In [28]:
print(df_uer_ll.shape)
df_uer_ll.head()

(228, 4)


,C1,C1_NM,uer,period
0,1101,서울 종로구,4.0,2021 2nd half
1,1102,서울 중구,4.3,2021 2nd half
2,1103,서울 용산구,4.8,2021 2nd half
3,1104,서울 성동구,5.0,2021 2nd half
4,1105,서울 광진구,3.8,2021 2nd half


In [29]:
df_uer_ll.shape

(228, 4)

## 7. Employment to Population Ratio

In [30]:
df_epr_hl = request_data_kosis(url = config['epr']['hl_url'], target_period = '2021', variable_name = 'epr')
df_epr_ll = request_data_kosis(url = config['epr']['ll_url'], target_period = '202102', variable_name = 'epr')
df_epr_ll['period'] = '2021 2nd half'

In [31]:
print(df_epr_hl.shape)
df_epr_hl.head()

(17, 4)


,C1,C1_NM,epr,period
0,11,서울특별시,59.2,2021
1,21,부산광역시,56.2,2021
2,22,대구광역시,58.2,2021
3,23,인천광역시,61.3,2021
4,24,광주광역시,58.2,2021


In [32]:
print(df_epr_ll.shape)
df_epr_ll.head()

(228, 4)


,C1,C1_NM,epr,period
0,1101,서울 종로구,56.1,2021 2nd half
1,1102,서울 중구,55.9,2021 2nd half
2,1103,서울 용산구,59.5,2021 2nd half
3,1104,서울 성동구,56.4,2021 2nd half
4,1105,서울 광진구,57.5,2021 2nd half


## 8. Informal Employment

In [33]:
df_ie_hl = request_data_kosis(config['ie']['hl_url'], target_period = '2021', variable_name = 'ie')

In [34]:
print(df_ie_hl.shape)
df_ie_hl.head()

(17, 4)


,C1,C1_NM,ie,period
0,11,서울특별시,38.5,2021
1,21,부산광역시,41.1,2021
2,22,대구광역시,38.3,2021
3,23,인천광역시,37.1,2021
4,24,광주광역시,40.0,2021
